# Route Identification Over Time, Approach 2
See `05_route_identification_over_time.ipynb` for previous work.
<br>Determined that more data is needed than what is given from `fct_monthly_routes`. exploring using `dim_routes` to get feed_key, route short/long name/id/desc and other. 
<br>Cannot rely on feed_key as it is unstable over time. Next best is gtfs_schedule key, but absolute best is source_record_id.
<br>Start with running `make install_env` to install everything needed. `helpers`,`rt_dates`,`sched_gcs`
<br>Query data from warehouse, then use this snippet from `make_routes_gdf` from `geography_utils`.
* ddf["geometry"] = ddf.pt_array.apply(make_linestring)
    
<br> then save out as geo parquet to the `gtfs_schedule` folder in GCS (so versioning and history stays) using 
* utils.geoparquet_gcs_export(
                vp_gdf,
                SEGMENT_GCS,
                f"vp_{analysis_date}"
            )

<br>also add `route_long_name` to query. then see if we are able to see any changes in routes via:
* feed key
* name
* route id
* route short name
* route long name
* route desc
    

In [1]:
import pandas as pd
from segment_speed_utils import helpers
from segment_speed_utils.project_vars import SCHED_GCS
from shared_utils import rt_dates

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Googl

In [50]:
sep_date = rt_dates.DATES["sep2023"]
aug_date = rt_dates.DATES["aug2023"]

# grabbing additional dates
april_date = rt_dates.DATES["apr2023"]
may_date = rt_dates.DATES["may2023"]
june_date = rt_dates.DATES["jun2023"]
july_date = rt_dates.DATES["jul2023"]

In [52]:
# list of all the months
months_list = [sep_date, aug_date, april_date, may_date, june_date, july_date]

In [53]:
months_list

['2023-09-13',
 '2023-08-15',
 '2023-04-12',
 '2023-05-17',
 '2023-06-14',
 '2023-07-12']

In [54]:
# test loop of import_scheduled_trips, iterated through months_list

# empty list
month_trips = {}

for months in months_list:
    df = helpers.import_scheduled_trips(
        months,
        columns=[
            "feed_key",
            "name",
            "gtfs_dataset_key",
            "route_id",
            "route_short_name",
            "route_long_name",
            "route_desc",
        ],
        filters=[[("name", "==", "Sacramento Schedule")]],
        get_pandas=True,
    )
    month_trips[months] = df

In [55]:
month_trips.keys()

dict_keys(['2023-09-13', '2023-08-15', '2023-04-12', '2023-05-17', '2023-06-14', '2023-07-12'])

In [56]:
april = month_trips['2023-04-12']
may = month_trips['2023-05-17']
june = month_trips['2023-06-14']
july = month_trips['2023-07-12']
aug = month_trips['2023-08-15']
sep = month_trips['2023-09-13']

In [ ]:
# use this pseudo code to concat all the different months data
# pd.concat can stack DFs on top of others.
# pd.concat([df1, df2], axis=0)

In [57]:
#test to see if i can use a dictionary in concat
all_trips = pd.concat([april,may,june,july,aug,sep],axis=0)

In [58]:
all_trips

,feed_key,name,schedule_gtfs_dataset_key,route_id,route_short_name,route_long_name,route_desc
0,55c36112e9a6a849c0ddc65eb54de7f1,Sacramento Schedule,43a1e46d592a1ee647bce8422c68460c,001,1,GREENBACK,None
1,55c36112e9a6a849c0ddc65eb54de7f1,Sacramento Schedule,43a1e46d592a1ee647bce8422c68460c,011,11,NATOMAS/LAND PARK,None
2,55c36112e9a6a849c0ddc65eb54de7f1,Sacramento Schedule,43a1e46d592a1ee647bce8422c68460c,013,13,NATOMAS/ARDEN,None
3,55c36112e9a6a849c0ddc65eb54de7f1,Sacramento Schedule,43a1e46d592a1ee647bce8422c68460c,015,15,DEL PASO HEIGHTS,None
4,55c36112e9a6a849c0ddc65eb54de7f1,Sacramento Schedule,43a1e46d592a1ee647bce8422c68460c,019,19,RIO LINDA,None
...,...,...,...,...,...,...,...
61,294252b3b4b42fbb31a31ce184fcb3f5,Sacramento Schedule,cb3074eb8b423dfc5acfeeb0de95eb82,227,227,SOUTH LAND PARK - GREENHAVEN DR,None
62,294252b3b4b42fbb31a31ce184fcb3f5,Sacramento Schedule,cb3074eb8b423dfc5acfeeb0de95eb82,228,228,GLORIA DR - RUSH RIVER DR,None
63,294252b3b4b42fbb31a31ce184fcb3f5,Sacramento Schedule,cb3074eb8b423dfc5acfeeb0de95eb82,247,247,21ST ST - FLORIN RD,None
64,294252b3b4b42fbb31a31ce184fcb3f5,Sacramento Schedule,cb3074eb8b423dfc5acfeeb0de95eb82,252,252,FREEPORT - FRUITRIDGE - ML KING,None
